In [ ]:

import requests 
from bs4 import BeautifulSoup
from pprint import pp
URL = 'https://books.toscrape.com/'
def explora_categoria(url):
    """ A partir de la URL de la página principal de una categoría, devuelve el nombre
        de la categoría y el número de libros """
    category_request=requests.get(url)
    # Corregimos el encoding
    category_request.encoding = category_request.apparent_encoding
    html = category_request.text
    soup = BeautifulSoup(html, 'html.parser')
    #Se ha visto el patron de que siempre
    #siempre el 1º strong es el nombre
    #y el segundo cuantos libros hay en total
    etiquetas=soup.find_all("strong")
    return (etiquetas[0].text,int(etiquetas[1].text))

def categorias():
    """ Devuelve un conjunto de parejas (nombre, número libros) de todas las categorías """
    r = requests.get(URL)
    # Corregimos el encoding
    r.encoding = r.apparent_encoding
    html = r.text
    soup = BeautifulSoup(html, 'html.parser')
    etiquetas = soup.find_all("div",{'class':"side_categories"})
    enlaces=etiquetas[0].find_all("a")
    #Descartamos la primera porque siempre es books
    return {explora_categoria(URL+enlace.attrs["href"])for enlace in enlaces[1:]}
    


con=categorias()
for libro_actual in con:
    print(libro_actual)


In [16]:
# APARTADO 2 #
import requests 
from bs4 import BeautifulSoup
from urllib.parse import urljoin
URL = 'https://books.toscrape.com/'
def url_categoria(nombre):
    """ Devuelve la URL de la página principal de una categoría a partir de su nombre (ignorar
        espacios al principio y final y también diferencias en mayúsculas/minúsculas) """
    r = requests.get(URL)
    # Corregimos el encoding
    r.encoding = r.apparent_encoding
    html = r.text
    soup = BeautifulSoup(html, 'html.parser')
    #https://www.geeksforgeeks.org/beautifulsoup-search-by-text-inside-a-tag/
    etiquetas = soup.find_all(lambda tag: tag.name=="a" and tag.text.strip().lower()==nombre.strip().lower())
    
    if(len(etiquetas)==0):
        return None
    else:
        return URL+etiquetas[0].attrs["href"]


def number_from_text(texto):
    """
    Devuelve el numero que representa el texto 
    que llega por parametro
    """
    numeros={
        'one':1,
        'two':2,
        'three':3,
        'four':4,
        'five':5
    }
    return numeros.get(texto.lower(),None)

def libros_de_la_pagina(soup):
    """
    Devuelve un conjunto con todos los libros que se ven 
    """
    etiquetas=soup.find_all("article",{'class':"product_pod"})
    libros=set()
    for etiqueta in etiquetas:
        #Siempre las estrellas son la primera P
        p=etiqueta.find("p")
        # Esta p no es la misma que la primera
        precio=etiqueta.find("p",{'class':"price_color"}).text[1:]
        titulo=etiqueta.find("h3").find("a").attrs["title"]
        libro_actual=(titulo,float(precio),number_from_text(p.attrs["class"][1]))
        libros.add(libro_actual)
    return libros

def todas_las_paginas(url):
    """ Sigue la paginación recopilando todas las URL *absolutas* atravesadas
     y los libros que se encuentran en esta """
    lista=[url]
    r = requests.get(url)
    r.encoding = r.apparent_encoding  # Corregimos el encoding
    html = r.text
    soup = BeautifulSoup(html, 'html.parser')
    conjunto=libros_de_la_pagina(soup)
    boton_siguiente=soup.find("li",{'class':"next"})
    if(boton_siguiente is None):
        return lista, conjunto
    else:
        pagina_siguiente=boton_siguiente.find("a").attrs["href"]
        siguiente_url=urljoin(url,pagina_siguiente)
        return lista+todas_las_paginas(siguiente_url)[0],conjunto |todas_las_paginas(siguiente_url)[1]

def libros_categoria(nombre):
    """ Dado el nombre de una categoría, devuelve un conjunto de tuplas 
        (titulo, precio, valoracion), donde el precio será un número real y la 
        valoración un número natural """
    cat_url=url_categoria(nombre)
    if cat_url is None:
        return set()
    else:
       return todas_las_paginas(cat_url)[1]
    
aux=libros_categoria("fiction")
for l in aux:
    print(l)
print(len(aux))

('The Testament of Mary', 52.67, 4)
('The Vacationers', 42.15, 4)
('Dear Mr. Knightley', 11.21, 5)
('Digital Fortress', 58.0, 5)
('The Dinner Party', 56.54, 2)
('My Name Is Lucy Barton', 41.56, 1)
('The Firm', 45.56, 3)
('Thirst', 17.27, 5)
('The Silent Sister (Riley MacPherson #1)', 46.29, 5)
('A Man Called Ove', 39.72, 1)
('Eligible (The Austen Project #4)', 27.09, 3)
('Three-Martini Lunch', 23.21, 3)
('Last One Home (New Beginnings #1)', 59.98, 3)
('The Course of Love', 16.78, 3)
('Take Me with You', 45.21, 3)
('Lila (Gilead #3)', 12.47, 3)
('I Am Pilgrim (Pilgrim #1)', 10.6, 4)
('Lies and Other Acts of Love', 45.14, 1)
('Mothering Sunday', 13.34, 2)
('Hystopia: A Novel', 21.96, 4)
('The Da Vinci Code (Robert Langdon #2)', 22.96, 2)
('The Art of Fielding', 22.1, 1)
('Deception Point', 40.32, 4)
('The High Mountains of Portugal', 51.15, 1)
('The First Hostage (J.B. Collins #2)', 25.85, 3)
('Mr. Mercedes (Bill Hodges Trilogy #1)', 28.9, 1)
("The Husband's Secret", 52.51, 5)
('Siddhart